Important notes:
1. python must be 3.8.x
2. if you get "libtinfo.so.5" error, do "sudo apt install libncurses5" - also see this link https://github.com/ChrisCummins/ProGraML/issues/187

What we need to do:
- This program takes all the cpp files int the HLS folder, and converts it into a networkx graph. graph_list is a list of networkx graphs, If a cpp files requires a header that is not in the standard library, you need to include that .h file in that folder.
- We have two important tasks before we can start applying it to a GNN model:
    - embed the node information into a vector. (I think arvind has a picture of a board stefan drew on that explains it well)
    - convert the MultiDiGraph into a normal Digraph I think


In [1]:
!python3 --version

Python 3.8.20


In [2]:
%pip install -U programl

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install protobuf==3.20.*
%pip install dgl==0.6.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import programl as pg
import numpy as np

Using backend: pytorch
/export/hdd/scratch/aramacha32/conda/envs/graph/lib/python3.8/site-packages/dgl/backend/pytorch/sparse.py:85: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/export/hdd/scratch/aramacha32/conda/envs/graph/lib/python3.8/site-packages/dgl/backend/pytorch/sparse.py:94: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dZ):
/export/hdd/scratch/aramacha32/conda/envs/graph/lib/python3.8/site-packages/dgl/backend/pytorch/sparse.py:147: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/export/hdd/scratch/aramacha32/conda/envs/graph/lib/python3.8/site-packages/dgl/backend/pytorch/sparse.py:156: FutureWarning: `t

In [11]:
import os
cpp_files = []
hls_reports = []
res = "/usr/scratch/ssam/result"
for root, dirs, files in os.walk(res):
    for folder in dirs:
        folder_path = res + "/" + folder
        for r, d, files in os.walk(folder_path):
            for file in files:
                #print(r)
                #print(r[25:28])
                if(r[25:28] == "ata"):
                    if file.endswith(".cpp"):
                        cpp_files.append([os.path.join(r, file)])
                    if file.endswith(".json"):
                        hls_reports.append(os.path.join(r, file))
print(cpp_files[0:3])
print(hls_reports[0:3])
print(len(cpp_files))

cpp_files = cpp_files[0:100]
hls_reports = hls_reports[0:100]


[['/usr/scratch/ssam/result/atax_opt_36b6a5399fb21f370bdb63809fb4e2ba/atax_hls.cpp'], ['/usr/scratch/ssam/result/atax_opt_8df7f2f6bfcce5e6c5fbcbdfd0467640/atax_hls.cpp'], ['/usr/scratch/ssam/result/atax_opt_c782838d48aa0f7ae38b685410bfa894/atax_hls.cpp']]
['/usr/scratch/ssam/result/atax_opt_36b6a5399fb21f370bdb63809fb4e2ba/data_hls.json', '/usr/scratch/ssam/result/atax_opt_8df7f2f6bfcce5e6c5fbcbdfd0467640/data_hls.json', '/usr/scratch/ssam/result/atax_opt_c782838d48aa0f7ae38b685410bfa894/data_hls.json']
676


In [6]:
#if you get the libtinfo.so.5 error, this should work: $sudo apt install libncurses5


In [13]:


Generator = pg.from_clang(cpp_files)

graph_list = []
t = 0
for graph in Generator:
    graph = pg.to_networkx(graph)
    graph_list.append(graph)
   
#print(len(graph_list))



In [14]:
#labels
import json

def parse_json(file_path, key):
    with open(file_path, "r") as file:
        data = json.load(file)

        if key in data:
            return data[key]
        else:
            print(f"Key '{key}' not found in the JSON file.")
            return None

labels = []

for file in hls_reports:
    avg_cycles = parse_json(file, "latency_average_cycles")
    resources_lut = parse_json(file, "resources_lut_used")
    label = [resources_lut]
    labels.append(label)
labels = np.array(labels)
print(labels.shape)

(100, 1)


In [74]:
#This is what Arvind made - so if you have any questions about it ask him - but this does all the node embeddings.
#It takes in every networkx graph in our list and represents them numerically (both nodes and edges)
#we may want to add more things to text_list - that list is the list of LLVM IR commands. that will not be very hard though


texts = []
x_features = []
x_features_list = []
text_list = ['icmp', 'i1','add','alloca','float','[64 x float]*','sext','getelementptr','float*','i64','store','br','i32','load','i32*']
print(len(text_list))
text = {}
edge_attr_list = []
edge_idx_list = []
for g in graph_list:
    for n in g.nodes(data=True):
        x = np.zeros(len(text_list) + 4, dtype='float64')
        dict = n[1]
        key = dict['text']
        #if key not in text_list:
        #    key = 'other'
        #    x[x.shape[0]-1] = 1
        #else:
        #    x[3 + text_list.index(key)] = 1
        x[0] = n[0]
        x[1] = dict['function']
        x[2] = dict['type']
        if key in text:
            text[key] += 1
        else:
            text[key] = 1
        #x_features.append({'x' : x})
        x_features.append(x)
    x_features_list.append(np.array(x_features, dtype='float64'))
    texts.append(text)
    edge_indexes = [[], []]
    edge_attr = []
    pairs = []
    for e in g.edges(data=True):
        if (e[0], e[1]) in pairs:
            continue
        pairs.append((e[0], e[1]))
        edge_indexes[0].append(e[0])
        edge_indexes[1].append(e[1])
        attr = e[2].values()
        edge = []
        for a in attr:
            edge.append(a)
        edge_attr.append(edge)
    edge_attr_list.append(np.array(edge_attr, dtype='int64'))
    edge_idx_list.append(edge_indexes)

15


In [75]:
sorted_text = sorted(text.items(), key= lambda x:x[1])
print(type(sorted_text))
print(sorted_text[-len(text_list):])
print(len(graph_list))

<class 'list'>
[('icmp', 700), ('i1', 700), ('add', 700), ('alloca', 900), ('float', 1400), ('[64 x float]*', 1600), ('sext', 2000), ('getelementptr', 2000), ('float*', 2000), ('i64', 2100), ('store', 2400), ('br', 3500), ('i32', 4400), ('load', 4600), ('i32*', 4800)]
100


In [17]:
%pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.


In [76]:
#convert to data object, and then to dataset object - we make a dataloader later
#VERY IMPORTANT = right now labels are hardcoded as ones, which is stupid, we need to take these labes from the HLS reports

import torch
from torch_geometric.data import Data
#print(labels)
test_len = 8
idxs = np.arange(len(x_features_list), dtype='int')
test_idx = np.random.choice(idxs, size=test_len)
print(test_idx.dtype)
print(type(x_features_list))
test_x_features = []
test_edge_index = []
test_labels = []

for idx in test_idx:
    test_x_features.append(x_features_list[idx])
    test_edge_index.append(edge_idx_list[idx])
    test_labels.append(labels[idx])
    
print(test_idx)
#print(test_x_features)
#print(test_edge_index)

#should probably assert x_features_list and edge_attr_list have same length
train_dataset = []
test_dataset = []
for i in range(0, len(x_features_list)): #len(x_features_list) = how many graphs we have (how many HLS programs)
    edge_index = torch.tensor(edge_idx_list[i]) #turns out you do need this tranpsose or else it gets mad...
    nodes = torch.tensor(x_features_list[i])
    #label = label_list[i] --- we need to make a label list
    label = labels[i]
    data = Data(x=nodes, edge_index = edge_index, y = torch.tensor(np.array([labels[i, :]])).T.double()) #maybe y in here is the labes I think... should probably replcae that with what we actually want, but just have it in for now as 1
    data.validate(raise_on_error=True)
    if i in test_idx:
        test_dataset.append(data)
    else:
        train_dataset.append(data)
print(len(train_dataset))
print(len(test_dataset))

int64
<class 'list'>
[44 15 84 21 76 49 22 41]
92
8


In [77]:
#not important, just prints out stuff...not super nessacary
data = train_dataset[0]

print(f'Number of features: {data.num_features}') #Number of features in the graph
print(f'Number of nodes: {data.num_nodes}') #Number of nodes in the graph
print(f'Number of edges: {data.num_edges}') #Number of edges in the graph
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}') # Average number of nodes in the graph
print(f'Contains isolated nodes: {data.has_isolated_nodes()}') #Does the graph contains nodes that are not connected
print(f'Contains self-loops: {data.has_self_loops()}') #Does the graph contains nodes that are linked to themselves
print(f'Is undirected: {data.is_undirected()}') #Is the graph an undirected graph

Number of features: 19
Number of nodes: 353
Number of edges: 575
Average node degree: 1.63
Contains isolated nodes: False
Contains self-loops: False
Is undirected: False


In [78]:
#this is the model I took form googel collab I linked in discord (for the most part)
#will want to play aorund with it a litlte bit
#IMPROTANT: FIGURE OUT WHAT WE ARE DOING WITH NUM_CLASSES

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import SAGPooling
from torch_geometric.nn import LayerNorm


num_node_features = train_dataset[0].num_node_features
num_classes = 1

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):

        hidden_linear_layer = 32

        super(GCN, self).__init__()
        torch.manual_seed(12345)
        print("!", num_node_features)
        self.conv1 = GCNConv(num_node_features, hidden_channels).double()
        self.conv2 = GCNConv(hidden_channels, hidden_channels).double()
        self.norm = LayerNorm(hidden_channels, mode='graph').double()
        self.conv3 = GCNConv(hidden_channels, hidden_channels).double()
        self.pool = SAGPooling(hidden_channels, ratio = 1.0).double()
        self.lin = Linear(hidden_channels, hidden_linear_layer).double()
        self.out = Linear(hidden_linear_layer, 1).double()

    def forward(self, x, edge_index):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index).double()
        x = x.relu()
        x = self.norm(x).double()
        x = self.conv2(x, edge_index).double()
        
        #x = x.relu()
        #x = self.conv3(x, edge_index).double()
        #print(x)
        x, pool_edge, _, _, attn_weights, _= self.pool(x, edge_index)
    
        x = torch.sum(x, dim = 0).view(1, -1).double()
        #print("!!")
        #print(x.shape)
        #print(x[0])
        #print("!!!")
        #print(x[1])
        # 3. Apply a final classifier
        x = self.lin(x).double()
        x = x.relu()
        x = self.out(x).double()
        
        return x.double()

model = GCN(hidden_channels=64)
print(model)

! 19
GCN(
  (conv1): GCNConv(19, 64)
  (conv2): GCNConv(64, 64)
  (norm): LayerNorm(64, affine=True, mode=graph)
  (conv3): GCNConv(64, 64)
  (pool): SAGPooling(GraphConv, 64, ratio=1.0, multiplier=1.0)
  (lin): Linear(in_features=64, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)


In [79]:
#create data loader for training - probably don't want batch size as 1

from torch_geometric.loader import DataLoader
train_loader =  DataLoader(train_dataset, batch_size=1, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=1)

In [42]:
!pip install torchmetrics

In [80]:
#TRAINING LOOP
#number of epochs is completely arbitrary
#TODO: SAVE THE MODEL somehow ( probably just copy code from HW)
from torchmetrics.regression import MeanAbsolutePercentageError

model = GCN(hidden_channels=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.L1Loss()


def train():
    model.train()

    loss = None
    out = None

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index)  # Perform a single forward pass.
         out = out.reshape((1, -1))
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         #torch.nn.utils.clip_grad_value_(model.parameters(), 0.1)
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.
    return loss, out

loss = None
out = None

for epoch in range(1, 15):
    loss, out = train()
    print("epoch: ", epoch, " loss: ", loss)





! 19
epoch:  1  loss:  tensor(1049.3853, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  2  loss:  tensor(854.9988, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  3  loss:  tensor(4264.7446, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  4  loss:  tensor(1485.2656, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  5  loss:  tensor(22800.4284, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  6  loss:  tensor(198588.2057, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  7  loss:  tensor(5563.3693, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  8  loss:  tensor(36885.5928, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  9  loss:  tensor(30851.7422, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  10  loss:  tensor(8004.5444, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  11  loss:  tensor(125989.4462, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  12  loss:  tensor(30467.0071, dtype=torch.float64, grad_fn=<MeanBackward0>)
e

In [81]:
l_acc = []
r_acc = []
for data in test_loader:
    out = model(data.x, data.edge_index)

    #predicted_latency_cycles = out.detach().numpy()[0,:]
    predicted_resources = out.detach().numpy()[0,:]

    #actual_latency = (data.y.detach().numpy().T)[:,0]
    actual_resources = (data.y.detach().numpy().T)[:,0]

    #latency_accuracy = 100 * abs((predicted_latency_cycles - actual_latency)/actual_latency)
    resources_accuracy = 100 * abs((predicted_resources - actual_resources) / actual_resources)

    #l_acc.append(latency_accuracy)
    r_acc.append(resources_accuracy)

    print("resources accuracy: ", resources_accuracy)

print(np.average(np.array(r_acc)), "average percent error")

resources accuracy:  [70.46904689]
resources accuracy:  [71.34625001]
resources accuracy:  [35.29729772]
resources accuracy:  [87.04116733]
resources accuracy:  [71.9633366]
resources accuracy:  [60.03850136]
resources accuracy:  [17.07169885]
resources accuracy:  [134.51343848]
68.46759215424174 average percent error
